In [ ]:
import transformers
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import re
import random
import datasets
from datasets import load_dataset
from abc import ABC, abstractmethod
from typing import List, Optional, Tuple, Dict
from tqdm import tqdm
import pickle
from dotenv import load_dotenv
import openai
import json

from typing import List, Optional, Tuple, Dict
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

import plotly.graph_objects as go
import plotly.express as px

from utils import untuple, eval_completions
from act_add.contrast_dataset import ContrastDataset
from scripts.get_activations import gen_pile_data, compare_token_lists, slice_acts
from act_add.model_wrapper import ModelWrapper
%load_ext autoreload
%autoreload 2